In [14]:
#dijkstra algo

import pygame, sys, random, math
from collections import deque
from tkinter import messagebox, Tk

# Initialize Pygame
size = (width, height) = 640, 480
pygame.init()
win = pygame.display.set_mode(size)
pygame.display.set_caption("Dijkstra’s Path Finding")
clock = pygame.time.Clock()

# Define grid dimensions
cols, rows = 64, 48
w = width // cols
h = height // rows
grid = []
queue, visited = deque(), []
path = []

# Spot class to represent each cell in the grid
class Spot:
    def __init__(self, i, j):
        self.x, self.y = i, j
        self.f, self.g, self.h = 0, 0, 0
        self.neighbors = []
        self.prev = None
        self.wall = False
        self.visited = False

    def show(self, win, col, shape=1):
        if self.wall:
            col = (0, 0, 0)
        if shape == 1:
            pygame.draw.rect(win, col, (self.x * w, self.y * h, w - 1, h - 1))
        else:
            pygame.draw.circle(win, col, (self.x * w + w // 2, self.y * h + h // 2), w // 3)

    def add_neighbors(self, grid):
        if self.x < cols - 1:
            self.neighbors.append(grid[self.x + 1][self.y])
        if self.x > 0:
            self.neighbors.append(grid[self.x - 1][self.y])
        if self.y < rows - 1:
            self.neighbors.append(grid[self.x][self.y + 1])
        if self.y > 0:
            self.neighbors.append(grid[self.x][self.y - 1])

# Handle clicking on the grid to make walls
def clickWall(pos, state):
    i = pos[0] // w
    j = pos[1] // h
    grid[i][j].wall = state

# Get position in the grid
def getPos(pos):
    i = pos[0] // w
    j = pos[1] // h
    return i, j

# Initialize the grid
for i in range(cols):
    arr = []
    for j in range(rows):
        arr.append(Spot(i, j))
    grid.append(arr)

for i in range(cols):
    for j in range(rows):
        grid[i][j].add_neighbors(grid)

# Main function
def main():
    flag = False
    noflag = True
    startflag = False
    start_set = False
    end_set = False
    start, end = None, None

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            # Handle mouse clicks for setting walls, start, and end points
            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                if not start_set:
                    start_pos = getPos(pos)
                    start = grid[start_pos[0]][start_pos[1]]
                    start.wall = False
                    start_set = True
                elif not end_set:
                    end_pos = getPos(pos)
                    end = grid[end_pos[0]][end_pos[1]]
                    end.wall = False
                    end_set = True
                else:
                    clickWall(pygame.mouse.get_pos(), True)

            if pygame.mouse.get_pressed()[2]:
                clickWall(pygame.mouse.get_pos(), False)

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN and start_set and end_set:
                    startflag = True
                    queue.append(start)
                    start.visited = True

        if startflag:
            if len(queue) > 0:
                current = queue.popleft()

                if current == end:
                    temp = current
                    while temp.prev:
                        path.append(temp.prev)
                        temp = temp.prev
                    if not flag:
                        flag = True
                        print("Done")
                else:
                    for i in current.neighbors:
                        if not i.visited and not i.wall:
                            i.visited = True
                            i.prev = current
                            queue.append(i)
            else:
                if noflag and not flag:
                    Tk().wm_withdraw()
                    messagebox.showinfo("No Solution", "There was no solution")
                    noflag = False

        # Drawing the grid and paths
        win.fill((0, 20, 20))
        for i in range(cols):
            for j in range(rows):
                spot = grid[i][j]
                spot.show(win, (44, 62, 80))
                if spot in path:
                    spot.show(win, (192, 57, 43))
                elif spot.visited:
                    spot.show(win, (39, 174, 96))
                if spot == start:
                    spot.show(win, (0, 255, 200))
                if spot == end:
                    spot.show(win, (0, 120, 255))
        
        pygame.display.flip()

# Run the program
main()


Done


SystemExit: 

In [18]:
# A* ago

import pygame, sys, math
from collections import deque
from tkinter import messagebox, Tk

# Initialize Pygame
size = (width, height) = 640, 640
pygame.init()
win = pygame.display.set_mode(size)
pygame.display.set_caption("A* Path Finding")
clock = pygame.time.Clock()

# Define grid dimensions
cols, rows = 64, 64
w = width // cols
h = height // rows
grid = []
open_set, closed_set = [], []
path = []

# Spot class to represent each cell in the grid
class Spot:
    def __init__(self, i, j):
        self.x, self.y = i, j
        self.f, self.g, self.h = 0, 0, 0
        self.neighbors = []
        self.prev = None
        self.wall = False
        self.visited = False

    def show(self, win, col, shape=1):
        if self.wall:
            col = (0, 0, 0)
        if shape == 1:
            pygame.draw.rect(win, col, (self.x * w, self.y * h, w - 1, h - 1))
        else:
            pygame.draw.circle(win, col, (self.x * w + w // 2, self.y * h + h // 2), w // 3)

    def add_neighbors(self, grid):
        if self.x < cols - 1:
            self.neighbors.append(grid[self.x + 1][self.y])
        if self.x > 0:
            self.neighbors.append(grid[self.x - 1][self.y])
        if self.y < rows - 1:
            self.neighbors.append(grid[self.x][self.y + 1])
        if self.y > 0:
            self.neighbors.append(grid[self.x][self.y - 1])

# Heuristic function (Manhattan distance)
def heuristic(a, b):
    return abs(a.x - b.x) + abs(a.y - b.y)

# Handle clicking on the grid to make walls
def clickWall(pos, state):
    i = pos[0] // w
    j = pos[1] // h
    grid[i][j].wall = state

# Get position in the grid
def getPos(pos):
    i = pos[0] // w
    j = pos[1] // h
    return i, j

# Initialize the grid
for i in range(cols):
    arr = []
    for j in range(rows):
        arr.append(Spot(i, j))
    grid.append(arr)

for i in range(cols):
    for j in range(rows):
        grid[i][j].add_neighbors(grid)

# Main function
def main():
    flag = False
    noflag = True
    startflag = False
    start_set = False
    end_set = False
    start, end = None, None

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            # Handle mouse clicks for setting walls, start, and end points
            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                if not start_set:
                    start_pos = getPos(pos)
                    start = grid[start_pos[0]][start_pos[1]]
                    start.wall = False
                    start_set = True
                elif not end_set:
                    end_pos = getPos(pos)
                    end = grid[end_pos[0]][end_pos[1]]
                    end.wall = False
                    end_set = True
                else:
                    clickWall(pygame.mouse.get_pos(), True)

            if pygame.mouse.get_pressed()[2]:
                clickWall(pygame.mouse.get_pos(), False)

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN and start_set and end_set:
                    startflag = True
                    open_set.append(start)

        if startflag:
            if len(open_set) > 0:
                current = min(open_set, key=lambda x: x.f)  # Get node with lowest f-score

                if current == end:
                    temp = current
                    while temp.prev:
                        path.append(temp.prev)
                        temp = temp.prev
                    if not flag:
                        flag = True
                        print("Path found")
                else:
                    open_set.remove(current)
                    closed_set.append(current)

                    for neighbor in current.neighbors:
                        if neighbor not in closed_set and not neighbor.wall:
                            temp_g = current.g + 1

                            if neighbor not in open_set:
                                open_set.append(neighbor)
                            elif temp_g >= neighbor.g:
                                continue

                            neighbor.g = temp_g
                            neighbor.h = heuristic(neighbor, end)
                            neighbor.f = neighbor.g + neighbor.h
                            neighbor.prev = current
            else:
                if noflag and not flag:
                    Tk().wm_withdraw()
                    messagebox.showinfo("No Solution", "There was no solution")
                    noflag = False

        # Drawing the grid and paths
        win.fill((0, 20, 20))
        for i in range(cols):
            for j in range(rows):
                spot = grid[i][j]
                spot.show(win, (44, 62, 80))
                if spot in path:
                    spot.show(win, (192, 57, 43))
                elif spot in closed_set:
                    spot.show(win, (39, 174, 96))
                if spot in open_set:
                    spot.show(win, (44, 62, 80))
                if spot == start:
                    spot.show(win, (0, 255, 200))
                if spot == end:
                    spot.show(win, (0, 120, 255))
        
        pygame.display.flip()

# Run the program
main()


SystemExit: 